In [2]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt


%matplotlib inline

sns.set_style("whitegrid")

In [3]:
df = pd.read_csv("housing.csv", encoding="utf-8")

c:\Users\Fibinse\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (6,15,17,19,20,21,22,23,24,25,26,28,29,30,31,32,33,34,35,37,38,39,40,41,42,43,44,46,47,48,49,50,51,52,55,56,57,58,59,60,61,64,65,66,67,68,69,70,74,126,127,128,143,144,164,166,168,169,170,171,172,173,174,175,177,178,179,180,181,182,183,184,186,187,188,189,190,191,192,193,195,196,197,198,199,200,201,204,205,206,207,208,209,210,213,214,215,216,217,218,219) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,Unnamed: 0,_id,building_name,building_uuid,city_select_uuid,city_select_uuid_0,city_select_uuid_1,completion_date,contact_persons_info_0_chat_enabled,contact_persons_info_0_contact_person_id,...,sellers_info_6_seller_label,sellers_info_6_uuid,show_loan_option,slot_id,status,street_info,thumb_image_url,title,type,updated_at
0,collection_ids,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,is_featured,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,is_uc_property,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,contact_persons_info,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,formatted_min_price,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df = pd.DataFrame({u'inventory_configs_0_apartment_type_id': 4, u'polygons_hash_h_district_name': None, u'parking_count': 1, u'polygons_hash_state_uuid': u'82b2a9c2e9b437c9ffdc', u'inventory_configs_0_property_type_id': 1, u'sellers_info_0_is_blocked': False, u'title': u'3 BHK Apartment', u'contact_persons_info_0_chat_enabled': None, u'polygons_hash_housing_region_uuid': u'd135fb588e8bb80e1fa2', u'polygons_hash_neighbourhood_name': None, u'contact_persons_info_0_profile_picture_url': u'https://housing.com/mystery-man.png', u'min_price': 31400000, u'contact_persons_info_0_is_blocked': False, u'inventory_configs_0_completion_date': 1425578517, u'inventory_configs_0_seller_0': 1, u'inventory_configs_0_area': 1850, u'sellers_info_0_name': u'Shamim', u'date_added': u'2016-03-04T23:31:57Z', u'inventory_configs_0_number_of_bedrooms': 3, u'sellers_info_0_contact_person_id': 1, u'sellers_info_0_profile_uuid': u'1deaa223-bbbd-4fa9-9ed9-aa4bab7484bb', u'polygons_hash_city_uuid': u'1ca99c33e3d8b987ccf1', u'display_locality_1': u'Andheri East', u'display_locality_0': u'Koldongri', u'formatted_min_price': u'3.14 Cr', u'inventory_canonical_url': u'/in/buy/resale/page/801351-3-bhk-apartment-in-andheri-east-for-rs-31400000', u'completion_date': 1425578517, u'impressions': 507, u'inventory_amenities_has_gym': True, u'building_name': u'Shivam Centrium', u'price_on_request': False, u'display_neighbourhood_0': u'Andheri East', u'display_neighbourhood_1': u'Koldongri', u'inventory_configs_0_price': 31400000, u'formatted_per_sqft_rate': u'16.97 k', u'inventory_amenities_has_gas_pipeline': True, u'polygons_hash_city_name': u'Mumbai', u'inventory_configs_0_number_of_toilets': 3, u'is_blocked': False, u'polygons_hash_housing_state_name': None, u'is_ad_property': None, u'inventory_configs_0_flat_config_id': 801351, u'sellers_info_0_profile_picture_url': u'https://housing.com/mystery-man.png', u'polygons_hash_housing_state_uuid': None, u'polygons_hash_state_name': u'Maharashtra', u'is_featured': False, u'is_uc_property': False, u'sellers_info_0_profile_url': u'1180304_shamim', u'building_uuid': u'31ebb4a552', u'polygons_hash_region_uuid': u'7318a08bb6570a26e893', u'date_added_in_seconds': 1457114517.294, u'thumb_image_url': u'https://is1-3.housingcdn.com/01c16c28/7e99a9651a8a14f421b3a16f341ca69c/v2/version.jpg', u'city_select_uuid_0': u'a0fd32816f73961748cf', u'inventory_configs_0_per_sqft_rate': 16972, u'contact_persons_info_0_profile_uuid': u'1deaa223-bbbd-4fa9-9ed9-aa4bab7484bb', u'inventory_configs_0_parking_count': 1, u'formatted_price': u'3.14 Cr', u'show_loan_option': True, u'type': u'resale', u'inventory_configs_0_id': 801351, u'contact_persons_info_0_profile_url': u'1180304_shamim', u'polygon_uuids_4': u'c81936bdf7593bb711dc', u'inventory_configs_0_is_available': True, u'inventory_configs_0_price_on_request': False, u'contact_persons_info_0_uuid': u'1deaa223-bbbd-4fa9-9ed9-aa4bab7484bb', u'inventory_amenities_has_lift': True, u'polygons_hash_housing_region_name': u'Western Suburbs', u'ctr': 0.04, u'location_coordinates': u'19.11288,72.848559', u'inventory_configs_0_formatted_price': u'3.14 Cr', u'formatted_max_price': u'3.14 Cr', u'polygons_hash_region_name': u'Mumbai Metropolitan Region', u'polygons_hash_locality_uuid': u'7d7e7ce92514320c5102', u'polygons_hash_sublocality_uuid': u'5f40a5d81efb8d0f3d07', u'inventory_amenities_is_gated_community': True, u'updated_at': 1488010088, u'street_info': u'Sahar Road, Near Vijay Nagar Hall, Koldongri', u'inventory_amenities_has_servant_room': False, u'id': 801351, u'polygon_uuids_5': u'3ac369d17698a4149caa', u'polygons_hash_neighbourhood_uuid': None, u'polygon_uuids_7': u'7318a08bb6570a26e893', u'polygon_uuids_6': u'a0fd32816f73961748cf', u'polygon_uuids_1': u'7d7e7ce92514320c5102', u'polygon_uuids_0': u'5f40a5d81efb8d0f3d07', u'polygon_uuids_3': u'd135fb588e8bb80e1fa2', u'polygon_uuids_2': u'f7f5d7f50dde9452144e', u'polygon_uuids_9': u'1ca99c33e3d8b987ccf1', u'polygon_uuids_8': u'82b2a9c2e9b437c9ffdc', u'developer_name': u'', u'slot_id': None, u'featured_type': u'locality', u'status': u'ACTIVE', u'inventory_amenities_has_swimming_pool': False, u'polygons_hash_h_district_uuid': None, u'polygons_hash_sublocality_name': u'Koldongri', u'price': 31400000, u'polygons_hash_locality_name': u'Andheri East', u'contact_persons_info_0_contact_person_id': 1, u'sellers_info_0_chat_enabled': None, u'contact_persons_info_0_name': u'Shamim', u'inventory_amenities_has_parking': True, u'inventory_configs_0_facing': u'north-east', u'_id': u'801351_resale', u'max_price': 31400000, u'sellers_info_0_uuid': u'1deaa223-bbbd-4fa9-9ed9-aa4bab7484bb'})

ValueError: If using all scalar values, you must pass an index

In [8]:
pd.concat?